In [1]:
%pip install sentence_transformers ipywidgets einops datasets

Note: you may need to restart the kernel to use updated packages.


In [13]:
from sentence_transformers import SentenceTransformer
from pathlib import Path
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display
from collections import defaultdict
from tqdm import tqdm

In [3]:
# Load https://huggingface.co/sentence-transformers/all-mpnet-base-v2
cache_folder = Path.home() / "tmp_cache"
model = SentenceTransformer("all-mpnet-base-v2", cache_folder=cache_folder)

In [4]:
embeddings = model.encode([
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
])

In [5]:
similarities = model.similarity(embeddings, embeddings)
similarities

tensor([[1.0000, 0.6817, 0.0492],
        [0.6817, 1.0000, 0.0421],
        [0.0492, 0.0421, 1.0000]])

Top original model for semantic search (see https://www.sbert.net/docs/sentence_transformer/pretrained_models.html)
multi-qa-mpnet-base-dot-v1

Multilingual semantic similarity model
distiluse-base-multilingual-cased-v1
https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v1

Instructor model (trained with instructions in mind)
hkunlp/instructor-large
https://huggingface.co/hkunlp/instructor-large

NV-Embed-v2
https://huggingface.co/nvidia/NV-Embed-v2

# Generate test data

In [6]:
# Sample data pools
first_names = ["Alice", "Bob", "Carlos", "Dmitry", "Emma", "Fatima", "George", "Hiroshi", "Isla", "Juan", "Katerina", "Luis", "Mikhail", "Natalia", "Omar", "Paula", "Quentin", "Ravi", "Sofia", "Tariq", "Ursula", "Victor", "Wei", "Xenia", "Yusuf", "Zara"]
last_names = ["Anderson", "Brown", "Chen", "Diaz", "Evans", "Fernández", "Gonzalez", "Hernandez", "Ivanov", "Johnson", "Kowalski", "Lopez", "Martinez", "Novikov", "O'Connor", "Petrov", "Quintero", "Rodriguez", "Sanchez", "Taylor", "Ulyanov", "Vega", "Wang", "Xiao", "Yamamoto", "Zhukov"]
job_titles = ["Data Scientist", "Software Engineer", "ML Engineer", "AI Researcher", "Financial Analyst", "Product Manager", "DevOps Engineer", "Cloud Architect", "Security Specialist", "QA Engineer", "Frontend Developer", "Backend Developer"]
seniorities = ["Junior", "Senior", "Lead", "Principal", "Staff", "Intern"]
locations = ["New York, NY", "San Francisco, CA", "London, UK", "Madrid, España", "Berlin, DE", "Москва, Россия", "Paris, FR", "Home Office", "HQ Building 3", "Sao Paulo, BR", "Toronto, CA"]
departments = ["AI", "Eng", "Finanzas", "R&D", "IT", "DevOps", "Infra", "Data", "CyberSec", "Маркетинг", "BizOps"]

# Function to introduce small variations (e.g., typos, abbreviations)
def introduce_variation(text):
    variations = [
        lambda t: t.replace("Engineer", "Eng."),  # Abbreviation
        lambda t: t.replace("Data", "D."),  # Shorten "Data"
        lambda t: t.lower() if random.random() < 0.1 else t,  # Random lowercase
        lambda t: t + " (remote)" if "Home Office" in text and random.random() < 0.5 else t,  # Clarify remote work
        lambda t: t.replace("Senior", "Sr.") if "Senior" in t else t,  # Abbreviate Senior
        lambda t: t + " " + random.choice(["Jr.", "II", "III"]) if "Intern" in t and random.random() < 0.3 else t,  # Add suffix to Interns
        lambda t: t.replace("Analyst", "Analista") if "Analyst" in t and random.random() < 0.2 else t,  # Spanish variation
        lambda t: t.replace("Manager", "Mgr.") if "Manager" in t and random.random() < 0.2 else t,  # Shorten Manager
        lambda t: t[:-1] + random.choice("xyz") if len(t) > 6 and random.random() < 0.05 else t,  # Introduce small typo
    ]
    return random.choice(variations)(text)

# Generate dataset
employees = []
for _ in range(100):
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    name = f"{first_name} {last_name}"
    job_title = introduce_variation(f"{random.choice(seniorities)} {random.choice(job_titles)}")
    location = introduce_variation(random.choice(locations))
    department = introduce_variation(random.choice(departments))

    employees.append({"name": name, "job_title": job_title, "location": location, "department": department})

df = pd.DataFrame(employees)
df

,name,job_title,location,department
0,Bob Hernandez,Senior QA Engineer,"Berlin, DE",Infra
1,Zara Diaz,Junior DevOps Engineer,"Berlin, DE",R&D
2,Hiroshi Hernandez,Principal Backend Developer,"London, UK",Infra
3,Luis Novikov,Intern Data Scientist,"Berlin, DE",Маркетинг
4,Carlos Petrov,Principal AI Researcher,"New York, NY",R&D
...,...,...,...,...
95,Juan Zhukov,Staff Data Scientist,"Москва, Россия",IT
96,Juan Ivanov,Principal Software Engineer,"New York, NY",DevOps
97,Ravi Gonzalez,Junior Financial Analyst,"Москва, Россия",Infra
98,Ursula Quintero,Intern Backend Developer,"Sao Paulo, Bx",Маркетинг


In [7]:
df['text'] = "Job title: " + df['job_title'] + "\nLocation: " + df['location'] + "\nDepartment: " + df['department']

In [8]:
print(df['text'][0])

Job title: Senior QA Engineer
Location: Berlin, DE
Department: Infra


In [9]:
questions = [
    "Who are the senior engineers working in NY?",
    "Find me all data scientists in London HQ",
    "People in AI dept. working remotely?",
    "Is there any anaylst in Madrid?",
    "Show all engs in SF office",
    "Lista de empleados en Finanzas en México",
    "Кто работает в отделе ИИ в Москве?",
    "Software devs in EU, any location",
    "Any Sr. Analyst in the Finance dept?",
    "Who’s working in AI at the Berlin office?",
    "Data scientists in remote, any country",
    "List of engineers in building 12?",
    "Can you find me a ML eng in US?",
    "Who are the people working in IT at Paris HQ?",
    "Finance folks in LatAm region?",
    "Empleados en el depto de ingeniería en Argentina",
    "Senior managers in tech, UK or Germany",
    "All ppl in eng, except remote",
    "Кто в Лондоне в отделе Финансов?",
    "Are there any junior devs in Madrid?"
]

In [10]:
model_names = [
    "multi-qa-mpnet-base-dot-v1",
    "distiluse-base-multilingual-cased-v1",
    "hkunlp/instructor-large",
    # "nvidia/NV-Embed-v2", # This model requires a lot of memory
    "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
]

models = {name: SentenceTransformer(name, cache_folder=cache_folder, trust_remote_code=True) for name in model_names}

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/145k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [19]:
def get_top_matches():
    top_matches = defaultdict(dict)
    for model_name, model in tqdm(models.items()):
        # Encode the questions and the text column
        question_instruction = "Represent search query for retrieval of employee contacts: " if model_name == "hkunlp/instructor-large" else None
        question_embeddings = model.encode(questions, prompt=question_instruction, show_progress_bar=True)

        text_instruction = "Represent employee attributes for retrieval: " if model_name == "hkunlp/instructor-large" else None
        text_embeddings = model.encode(df['text'].tolist(), prompt=text_instruction, show_progress_bar=True)

        # Compute cosine similarity
        cosine_similarities = cosine_similarity(question_embeddings, text_embeddings)

        # Retrieve top 5 matches for each question
        for i, question in enumerate(questions):
            top_indices = np.argsort(cosine_similarities[i])[::-1][:5]
            top_similarities = cosine_similarities[i][top_indices]
            top_matches[question][model_name] = (df.iloc[top_indices], top_similarities)

    return top_matches

top_matches = get_top_matches()

  0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:05<00:17,  5.77s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

 75%|███████▌  | 3/4 [00:17<00:05,  5.98s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [03:24<00:00, 51.15s/it]


In [21]:
def display_top_matches(top_matches):
    # Display the results
    for question, results in top_matches.items():
        print(f"Question: {question}\n")
        for model_name, (matches, similarities) in results.items():
            print(f"Model: {model_name}\n")
            matches = matches.copy()
            matches['cosine_similarity'] = similarities
            display(matches[['name', 'job_title', 'location', 'department', 'cosine_similarity']].style.set_table_styles(
                [{'selector': 'th', 'props': [('font-size', '120%'), ('text-align', 'center')]},
                {'selector': 'td', 'props': [('font-size', '110%'), ('text-align', 'center')]}]
            ).set_properties(**{'background-color': 'lightyellow'}, subset=['cosine_similarity']))
        print("\n" + "="*80 + "\n")

display_top_matches(top_matches)

Question: Who are the senior engineers working in NY?

Model: multi-qa-mpnet-base-dot-v1



,name,job_title,location,department,cosine_similarity
39,Victor Novikov,Lead ML Engineer,"New York, NY",R&D,0.683339
96,Juan Ivanov,Principal Software Engineer,"New York, NY",DevOps,0.668728
10,Natalia Chen,Senior ML Engineer,Home Office,IT,0.658557
12,Ravi Quintero,Staff Backend Developer,"New York, NY",Infra,0.640238
92,Victor Diaz,Junior ML Engineer,Home Office,IT,0.627096


Model: distiluse-base-multilingual-cased-v1



,name,job_title,location,department,cosine_similarity
39,Victor Novikov,Lead ML Engineer,"New York, NY",R&D,0.516727
96,Juan Ivanov,Principal Software Engineer,"New York, NY",DevOps,0.505807
44,Luis O'Connor,Staff DevOps Engineer,"San Francisco, CA",Eng,0.417064
89,George Novikov,Senior DevOps Engineer,"Москва, Россия",IT,0.413501
94,Natalia Hernandez,Staff DevOps Engineer,"Toronto, CA",BizOps,0.410772


Model: hkunlp/instructor-large



...

,name,job_title,location,department,cosine_similarity
45,Ravi Anderson,Intern Frontend Developer,"Madrid, España",R&D,0.389803
86,Dmitry Sanchez,Junior Cloud Architect,"Madrid, España",BizOps,0.387820
37,Isla Chen,Junior Frontend Developer,"London, UK",R&D,0.263690
1,Zara Diaz,Junior DevOps Engineer,"Berlin, DE",R&D,0.262512
93,Alice Johnson,Staff Financial Analyst,"Madrid, España",Маркетинг,0.259075


Model: hkunlp/instructor-large



,name,job_title,location,department,cosine_similarity
45,Ravi Anderson,Intern Frontend Developer,"Madrid, España",R&D,0.925800
86,Dmitry Sanchez,Junior Cloud Architect,"Madrid, España",BizOps,0.920603
93,Alice Johnson,Staff Financial Analyst,"Madrid, España",Маркетинг,0.894144
98,Ursula Quintero,Intern Backend Developer,"Sao Paulo, Bx",Маркетинг,0.885054
76,Alice O'Connor,Staff Frontend Developer,"Berlin, DE",AI,0.884418


Model: Alibaba-NLP/gte-Qwen2-1.5B-instruct



,name,job_title,location,department,cosine_similarity
86,Dmitry Sanchez,Junior Cloud Architect,"Madrid, España",BizOps,0.561860
45,Ravi Anderson,Intern Frontend Developer,"Madrid, España",R&D,0.526486
1,Zara Diaz,Junior DevOps Engineer,"Berlin, DE",R&D,0.495154
37,Isla Chen,Junior Frontend Developer,"London, UK",R&D,0.473749
17,Juan Xiao,Junior Frontend Developer,"Москва, Россия",Infra,0.456811


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from IPython.display import display

# Encode the questions and the text column
question_embeddings = model_multi_qa.encode(questions)
text_embeddings = model_multi_qa.encode(df['text'].tolist())

# Compute cosine similarity
cosine_similarities = cosine_similarity(question_embeddings, text_embeddings)

# Retrieve top 5 matches for each question
top_matches = {}
for i, question in enumerate(questions):
    top_indices = np.argsort(cosine_similarities[i])[::-1][:5]
    top_similarities = cosine_similarities[i][top_indices]
    top_matches[question] = (df.iloc[top_indices], top_similarities)

# Display the results
for question, (matches, similarities) in top_matches.items():
    matches = matches.copy()
    matches['cosine_similarity'] = similarities
    print(f"Question: {question}\n")
    display(matches[['name', 'job_title', 'location', 'department', 'cosine_similarity']].style.set_table_styles(
        [{'selector': 'th', 'props': [('font-size', '120%'), ('text-align', 'center')]},
         {'selector': 'td', 'props': [('font-size', '110%'), ('text-align', 'center')]}]
    ).set_properties(**{'background-color': 'lightyellow'}, subset=['cosine_similarity']))
    print("\n" + "="*80 + "\n")

Question: Who are the senior engineers working in NY?



,name,job_title,location,department,cosine_similarity
79,Yusuf Vega,Staff Frontend Developer,"New York, NY",Eng,0.688557
6,Emma Diaz,Staff ML Engineer,"New York, NY",Маркетинг,0.682026
72,Luis Taylor,Lead QA Engineer,"New York, NY",Infra,0.667823
42,Hiroshi Anderson,Lead Backend Developer,"New York, NY",Data,0.654799
95,Fatima Kowalski,Senior Product Managex,"New York, NY",AI,0.637155




Question: Find me all data scientists in London HQ



,name,job_title,location,department,cosine_similarity
9,Hiroshi Taylor,Intern Data Scientist,"London, UK",R&D,0.730971
44,Juan Zhukov,Junior Data Scientist,"London, UK",Data,0.712025
56,Bob Petrov,Staff DevOps Engineer,"London, UK",Data,0.658398
61,Paula Diaz,Staff Software Engineer,HQ Building 3,Data,0.646586
60,Isla Ivanov,Intern AI Researcher,"London, UK",Eng,0.641826




Question: People in AI dept. working remotely?



...

,name,job_title,location,department,cosine_similarity
88,Xenia Lopez,Junior Backend Developer,HQ Building 3,Eng,0.389830
21,George Hernandez,Lead QA Engineer,Home Office (remote),DevOps,0.379902
74,Ursula Novikov,Staff QA Engineer,Home Office,Маркетинг,0.360302
46,Omar Wang,Staff DevOps Engineer,Home Office,Eng,0.358982
36,Bob Chen,Staff Software Engineer,HQ Building 3,D.,0.356168




Question: Кто в Лондоне в отделе Финансов?



,name,job_title,location,department,cosine_similarity
57,Alice Quintero,Principal Frontend Developer,"Москва, Россия",DevOps,0.423991
26,Natalia Brown,Senior D. Scientist,"Москва, Россия",Infra,0.412721
41,Luis Ivanov,Staff AI Researcher,"Paris, FR",Маркетинг,0.401040
93,Victor Kowalski,junior data scientist,"Москва, Россия",CyberSec,0.396991
97,Carlos Johnson,Staff Cloud Architect,"Москва, Россия",DevOps,0.390706




Question: Are there any junior devs in Madrid?



,name,job_title,location,department,cosine_similarity
91,Sofia Sanchez,Junior Product Manager,"Madrid, España",Eng,0.641094
28,Wei Ivanov,Staff Data Scientist,"Madrid, España",CyberSec,0.555180
82,Alice Chen,Junior DevOps Engineer,"Berlin, DE",Infra,0.547909
92,George Petrov,Junior Frontend Developer,"Berlin, DE",R&D,0.540904
88,Xenia Lopez,Junior Backend Developer,HQ Building 3,Eng,0.529776
